In [1]:
import numpy as np
from acrobotics.util import get_default_axes3d
from acrobotics.recources.robots import Kuka
from acrobotics.path import FreeOrientationPt, TolOrientationPt
from acrobotics.geometry import Shape, Collection
from pyquaternion import Quaternion

In [2]:
q_nominal = Quaternion(axis=[1, 0, 0], angle=np.pi)
# a.transformation_matrix
tf_nominal = np.array([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1.0]])
tf_nominal

array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [3]:
robot = Kuka()

path = []
for s in np.linspace(0, 1, 8):
    xi = 0.8
    yi = s * 0.2 + (1-s) * (-0.2)
    zi = 0.2
    #path.append(FreeOrientationPt([xi, yi, zi]))
    path.append(TolOrientationPt([xi, yi, zi], q_nominal))

floor_plane = Shape(0.5, 0.5, 0.1)
floor_plane_tf = np.array([[1, 0, 0, 0.80],
                            [0, 1, 0, 0.00],
                            [0, 0, 1, 0.12],
                            [0, 0, 0, 1]])

# scene = Collection([floor_plane], [floor_plane_tf])
scene = Collection([], [])

In [4]:
from acrobotics.planning import cart_to_joint_no_redundancy
from acrobotics.planning import get_shortest_path

Q = cart_to_joint_no_redundancy(robot, path, scene, num_samples=200)

print([len(qi) for qi in Q])
qp = [qi[0] for qi in Q]

res = get_shortest_path(Q, method='dijkstra')
print(res)
qp_sol = res['path']

Processing point 0/8
sampling near with distance 0.1
Processing point 1/8
sampling near with distance 0.1
Processing point 2/8
sampling near with distance 0.1
Processing point 3/8
sampling near with distance 0.1
Processing point 4/8
sampling near with distance 0.1
Processing point 5/8
sampling near with distance 0.1
Processing point 6/8
sampling near with distance 0.1
Processing point 7/8
sampling near with distance 0.1
[1600, 1600, 1600, 1600, 1600, 1600, 1600, 1600]
{'success': True, 'path': [array([-0.24427849,  1.42337239, -0.31847784, -0.00568493, -1.08256698,
       -0.0765051 ]), array([-0.17458117,  1.43629885, -0.33335242, -0.01676018, -1.11323833,
       -0.08447085]), array([-0.10694712,  1.44994056, -0.34884718,  0.00165453, -1.10643363,
       -0.05510662]), array([-0.03663515,  1.45587325, -0.35554531,  0.00728013, -1.10845399,
       -0.05422201]), array([ 0.03466369,  1.456128  , -0.35583058,  0.00805881, -1.10703146,
        0.03357147]), array([ 0.10572308,  1.4496938

In [5]:
%matplotlib qt

#%% ANIMATE
fig2, ax2 = get_default_axes3d([-1, 1], [-1, 1], [-1, 1])
for pi in path: pi.plot(ax2)
scene.plot(ax2, c='g')
robot.animate_path(fig2, ax2, qp_sol)
plt.show(block=True)